In [2]:
from importlib import reload
from Traincenterv2 import readweaving, dataset
from Testcenterv2 import predtest
import Traincenterv2
import Testcenterv2
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
%load_ext autoreload
%autoreload 2

/home/haddad/anaconda3/envs/gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
image='noncompacted'#nom du .tif à lire (sans extension)
coordmatlab="noncompacted.mat"#fichier de coordonnée des fibres neutres (en .mat ou .vxl)
directiontrain="weft"
directiontest="warp"
resolution=1#conversion de la résolution si les proportion entre l'image et les coordonnées du fichiers ne correspondent pas

In [ ]:
Traincenterv2= reload(Traincenterv2)
from Traincenterv2 import readweaving
I,M=readweaving(image, directiontrain,coordmatlab=coordmatlab)#lecture de l'image et de ses coordonnée

# Hyperparameters

In [ ]:
#affichage de l'image pour vérifier le format et régler la vérité de terrain
from skimage.morphology import dilation, square
print(I.shape)
print(M.shape)
klayer=np.random.randint(I.shape[0])
size_fig = 16 #taille de la figure
size_neutral_fiber = 14 #taille des fibres dans la vérité de terrain qu'on génére
print(klayer)
plt.figure(figsize=(size_fig,size_fig))
plt.imshow(I[klayer,:,:],cmap='gray')
plt.figure(figsize=(size_fig,size_fig))
plt.imshow((1-dilation(M[klayer,:,:]>0,square(size_neutral_fiber))),cmap='gray')
plt.figure(figsize=(size_fig,size_fig))
plt.imshow((1-dilation(M[klayer,:,:]>0,square(size_neutral_fiber)))*I[klayer,:,:],cmap='gray')

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()#affichage des indentifiants des différents GPU disponible

In [ ]:
import keras
from keras.optimizers import Adam
from dlia_tools.u_net import u_net
from dlia_tools.keras_custom_loss import jaccard2_loss
# training params
nb_train_samples=10000
nb_val_samples=1000
batch_size=24
nb_epoch=45
# architecture params
nb_filters_0 = 4
sigma_noise = 0.01
img_channels=1
dim_patch_x=128 #dimensions des patchs constituant le set d'entrainement choisir un multiple 2
dim_patch_y=128 #dimensions des patchs constituant le set d'entrainement choisir un multiple 2
cuda="3" #id du GPU que l'on souhaite utiliser, si pas de GPU fixer la valeur à None
pas=1 #fréquence de slices annotées
# ****  train
opt_name = 'adam'  # choices:adadelta; sgd, rmsprop, adagrad, adam
loss_func = jaccard2_loss  #choices:mse, mae, binary_crossentropy, jaccard2_loss
if opt_name == "sgd":
    opt = SGD(lr=0.1)
elif opt_name == "rmsprop":
    opt = RMSprop()
elif opt_name == "adagrad":
    opt = Adagrad()
elif opt_name == "adadelta":
    opt = Adadelta()
elif opt_name == "adam":
    opt = Adam(lr=1e-5)
else:
    raise NameError("Wrong optimizer name")
# ****  deep learning model
shape = (dim_patch_x,dim_patch_y, img_channels)#Dimension pour entrainer les patchs (piste essayer d'entrainer avec un dataset de patch de taille variée)
model = u_net(shape, nb_filters_0, sigma_noise=sigma_noise)


# Training on Non compacted Weft Direction

In [ ]:
Traincenterv2 = reload(Traincenterv2)
from Traincenterv2 import traincenter
#Loss retournées sous forme de listes, penser à les sauvegarder si hors du notebook si on souhaite les conserver
val_loss, loss, modeldirectory, modelfile=traincenter(model, loss_func, opt,cuda,image,
                                                      coordmatlab, directiontrain, nb_train_samples, 
                                                      nb_val_samples, batch_size,dim_patch_x,
                                                      dim_patch_y, size_neutral_fiber,pas, nb_epoch)

# Test on Non compacted Warp Direction without evaluation

In [ ]:
directiontest="warp"
shape = (None, None, img_channels)#réglage pour que le réseau accepte toute les dimension d'image d'entrée
modelLarge = u_net(shape, nb_filters_0, sigma_noise=sigma_noise)
listDistFromGT,listDistFromDet,DetectionRate,GTfoundRate,Lcentroid,Lgt=predtest(modelLarge,loss_func, opt,
                                                                                modeldirectory,modelfile,image,
                                                                                coordmatlab, directiontest,resolution,
                                                                                cuda=cuda)

In [ ]:
I2,M2=readweaving(image, directiontest,coordmatlab=coordmatlab)
print(I2.shape)

In [ ]:
klayer=np.random.randint(I2.shape[0])
print(klayer)
plt.figure(figsize=(16,16))
plt.imshow(I2[klayer,:,:],cmap='gray')
plt.plot(Lcentroid[klayer][:,1],Lcentroid[klayer][:,0],'.g')
plt.plot(Lgt[klayer][:,1],Lgt[klayer][:,0],'.r')

In [4]:
pred = io.imread('prediction.tif')
result = io.imread('resultat.tif')

FileNotFoundError: [Errno 2] No such file or directory: '/home/haddad/ZYAD/github/U-net Neutral Fiber/prediction.tif'

In [ ]:
klayer=np.random.randint(I2.shape[0])
plt.figure(figsize=(16,16))
plt.imshow(pred[klayer,:,:],cmap='gray')
plt.figure(figsize=(16,16))
plt.imshow(result[klayer,:,:],cmap='gray')
plt.figure(figsize=(16,16))
plt.imshow((1-dilation(M2[klayer,:,:]>0,square(7)))*I2[klayer,:,:],cmap='gray')

In [ ]:
plt.figure()
plt.subplot(2,2,1)
plt.plot(np.array(DetectionRate),'ro')
plt.title('True Positive Rate (sensitivity) per slice')
plt.subplot(2,2,2)
plt.plot(np.array(GTfoundRate),'bo')
plt.title('Precision Rate per slice')
plt.subplot(2,2,3)
plt.title('Histogram of the mean distance difference')
plt.hist((np.array(listDistFromGT)[:,0]),density=True)
plt.subplot(2,2,4)
plt.title('Mean distance difference per slice')
plt.plot(np.array(listDistFromGT)[:,0])